# BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [21]:
!pip install langchain_google_genai

  Using cached langchain_google_genai-0.0.6-py3-none-any.whl (15 kB)
  Using cached google_generativeai-0.3.2-py3-none-any.whl (146 kB)
  Using cached google_ai_generativelanguage-0.4.0-py3-none-any.whl (598 kB)
  Using cached google_auth-2.27.0-py2.py3-none-any.whl (186 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 4.9 MB/s eta 0:00:00
  Using cached protobuf-4.25.2-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl (48 kB)
  Using cached googleapis_common_protos-1.62.0-py2.py3-none-any.whl (228 kB)
  Using cached cachetools-5.3.2-py3-none-any.whl (9.3 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl (84 kB)


In [7]:
!pip install --upgrade --quiet  duckduckgo-search

In [52]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [54]:
import os


## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [55]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [56]:
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [66]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", )
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [67]:
from langchain.globals import set_debug
set_debug(True)

In [59]:
llm = GoogleGenerativeAI(model='gemini-pro', )


In [68]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
# from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import DuckDuckGoSearchRun
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
# search = SerpAPIWrapper()
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [69]:
llm = OpenAI(temperature=0)
# llm = llm
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [70]:
llm_chain.get_prompts()

[]

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [71]:
OBJECTIVE = "Write a weather report for SF today"

In [74]:
# Logging of LLMChains
verbose = True
# If None, will keep on going forever
max_iterations: Optional[int] = 4
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [75]:
baby_agi({"objective": OBJECTIVE})

[chain/start] [1:chain:BabyAGI] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today"
}

*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list
[chain/start] [1:chain:BabyAGI > 2:chain:AgentExecutor] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today",
  "context": "Make a todo list\nGather current weather data for San Francisco: Use reliable sources to collect the most up-to-date weather data for San Francisco, including temperature, humidity, wind speed, and any other relevant information.\nAnalyze weather patterns and trends in San Francisco",
  "task": "Make a todo list"
}
[chain/start] [1:chain:BabyAGI > 2:chain:AgentExecutor > 3:chain:LLMChain] Entering Chain run with input:
{
  "objective": "Write a weather report for SF today",
  "context": "Make a todo list\nGather current weather data for San Francisco: Use reliable sources to collect the most up-to-date weather data for San Francisc

ValueError: Tried to add ids that already exist: {'result_1_0'}